In [14]:
from neo4j import GraphDatabase
import json
from pprint import pprint

import os
from dotenv import load_dotenv

from neo4j_graphrag.llm import OpenAILLM
from neo4j_graphrag.retrievers import Text2CypherRetriever


load_dotenv()

True

## 1. Connect to Neo4j

In [5]:
# Connection details
NEO4J_URL = os.getenv("NEO4J_URL")
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")

# Create driver
driver = GraphDatabase.driver(NEO4J_URL, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

# Test connection
with driver.session() as session:
    result = session.run("RETURN 1 as test")
    print("✓ Connection successful!")
    print(f"Test query result: {result.single()['test']}")

✓ Connection successful!
Test query result: 1


## 2. Get Database Schema

In [6]:
def get_schema(driver):
    """Fetch and display the database schema."""
    with driver.session() as session:
        # Get node labels
        labels_result = session.run("CALL db.labels()")
        labels = [record["label"] for record in labels_result]
        
        # Get relationship types
        rels_result = session.run("CALL db.relationshipTypes()")
        rel_types = [record["relationshipType"] for record in rels_result]
        
        print("=== Node Labels ===")
        for label in labels:
            print(f"  - {label}")
        
        print(f"\nTotal: {len(labels)} labels")
        
        print("\n=== Relationship Types ===")
        for rel in rel_types:
            print(f"  - {rel}")
        
        print(f"\nTotal: {len(rel_types)} relationship types")

get_schema(driver)

=== Node Labels ===
  - GraphNode
  - module
  - screen
  - value_object
  - form
  - form_field
  - session
  - function
  - view
  - button
  - route
  - action_type
  - event
  - flag
  - action
  - delegate
  - facade
  - product
  - dao
  - database_table
  - sql_query

Total: 21 labels

=== Relationship Types ===
  - CALLS
  - FORWARDS_TO
  - ACCESSES
  - DEPENDS_ON
  - uses_query
  - executed_by
  - accesses_table
  - uses_table
  - USES_ROUTE
  - DISPLAYS_VIEW
  - USES_FORM
  - CONTAINS_BUTTON
  - TRIGGERS_EVENT
  - INVOKES_ACTION_TYPE
  - ROUTES_TO_ACTION
  - MAPS_TO_ACTION
  - INITIALIZES_SCREEN
  - IMPLEMENTS_FUNCTION
  - FORWARDS_TO_ROUTE
  - MAPS_TO_VO
  - STORES_VO
  - CONTROLS_VISIBILITY
  - CONTROLS_STATE
  - STORED_IN_SESSION
  - INCLUDES_VIEW
  - DISPLAYS_SESSION_DATA
  - NAVIGATES_TO_SCREEN
  - RETURNS_TO_SCREEN
  - HAS_FIELD

Total: 29 relationship types


## 3. Get Database Statistics

In [7]:
def get_stats(driver):
    """Get basic statistics about the database."""
    with driver.session() as session:
        # Count nodes
        node_count = session.run("MATCH (n) RETURN count(n) as count").single()["count"]
        
        # Count relationships
        rel_count = session.run("MATCH ()-[r]->() RETURN count(r) as count").single()["count"]
        
        # Count by label
        label_counts = session.run("""
            CALL db.labels() YIELD label
            CALL {
                WITH label
                MATCH (n)
                WHERE label IN labels(n)
                RETURN count(n) as count
            }
            RETURN label, count
            ORDER BY count DESC
        """).data()
        
        print("=== Database Statistics ===")
        print(f"Total Nodes: {node_count}")
        print(f"Total Relationships: {rel_count}")
        print("\n=== Nodes by Label ===")
        for item in label_counts:
            print(f"  {item['label']}: {item['count']}")

get_stats(driver)

Received notification from DBMS server: <GqlStatusObject gql_status='01N00', status_description='warn: feature deprecated. CALL subquery without a variable scope clause is deprecated. Use CALL (label) { ... }', position=<SummaryInputPosition line=3, column=13, offset=54>, raw_classification='DEPRECATION', classification=<NotificationClassification.DEPRECATION: 'DEPRECATION'>, raw_severity='WARNING', severity=<NotificationSeverity.WARNING: 'WARNING'>, diagnostic_record={'_classification': 'DEPRECATION', '_severity': 'WARNING', '_position': {'offset': 54, 'line': 3, 'column': 13}, 'OPERATION': '', 'OPERATION_CODE': '0', 'CURRENT_SCHEMA': '/'}> for query: '\n            CALL db.labels() YIELD label\n            CALL {\n                WITH label\n                MATCH (n)\n                WHERE label IN labels(n)\n                RETURN count(n) as count\n            }\n            RETURN label, count\n            ORDER BY count DESC\n        '


=== Database Statistics ===
Total Nodes: 195
Total Relationships: 332

=== Nodes by Label ===
  GraphNode: 195
  database_table: 49
  form_field: 19
  sql_query: 17
  dao: 15
  flag: 11
  session: 10
  screen: 9
  button: 9
  action_type: 9
  event: 9
  function: 8
  value_object: 4
  route: 4
  delegate: 4
  facade: 4
  product: 4
  form: 3
  view: 3
  action: 3
  module: 1


## 4. Sample Node Properties

In [8]:
def show_sample_nodes(driver, label="GraphNode", limit=3):
    """Show sample nodes with their properties."""
    with driver.session() as session:
        result = session.run(f"""
            MATCH (n:{label})
            RETURN n
            LIMIT $limit
        """, limit=limit)
        
        print(f"=== Sample {label} Nodes ===")
        for i, record in enumerate(result, 1):
            node = record["n"]
            print(f"\nNode {i}:")
            print(f"  Labels: {list(node.labels)}")
            print(f"  Properties:")
            for key, value in dict(node).items():
                # Truncate long values
                if isinstance(value, str) and len(value) > 100:
                    value = value[:100] + "..."
                print(f"    {key}: {value}")

show_sample_nodes(driver)

=== Sample GraphNode Nodes ===

Node 1:
  Labels: ['screen', 'GraphNode']
  Properties:
    url_pattern: /dsmart/contract/keiyakuList/yuusyouKeiyakuListAssign.do
    action_class: YuusyouKeiyakuListAssignAction
    main_jsp: yuusyouKeiyakuListAssign.jsp
    description: Screen for creating after-sales contracts
    layout_type: form
    title: After Contract Screen
    source_file: ctc-data-en/contract-list/screen-flow-en.md
    access_level: authenticated_user
    uid: screen:after_contract_creation
    name: After Contract Creation Screen
    screen_id: GCNT19001
    form_bean: keiyaku_cardForm
    parent_module: module:contract_list

Node 2:
  Labels: ['screen', 'GraphNode']
  Properties:
    url_pattern: /dsmart/contract/keiyakuList/moushideSelectInit.do
    action_class: MoushideSelectInitAction
    main_jsp: moushideSelect.jsp
    description: Screen for selecting and creating multiple orders
    layout_type: selection_form
    title: Multiple Order Selection
    source_file: ctc

## 5. Test Text-to-Cypher Schema Generation

In [9]:
def get_detailed_schema(driver):
    """Get detailed schema suitable for Text2Cypher."""
    with driver.session() as session:
        # Get node type properties
        try:
            node_schema = session.run("""
                CALL db.schema.nodeTypeProperties()
                YIELD nodeType, propertyName, propertyTypes, mandatory
                RETURN nodeType, propertyName, propertyTypes, mandatory
            """).data()
        except:
            # Fallback for older Neo4j versions
            node_schema = []
        
        # Get relationship type properties
        try:
            rel_schema = session.run("""
                CALL db.schema.relTypeProperties()
                YIELD relType, propertyName, propertyTypes
                RETURN relType, propertyName, propertyTypes
            """).data()
        except:
            rel_schema = []
        
        print("=== Detailed Schema for Text2Cypher ===")
        print("\nNode Types and Properties:")
        if node_schema:
            for item in node_schema:
                mandatory = item.get('mandatory', False)
                marker = " *" if mandatory else ""
                print(f"  {item['nodeType']}.{item['propertyName']}: {item['propertyTypes']}{marker}")
        else:
            print("  (Schema introspection not available)")
        
        print("\nRelationship Types and Properties:")
        if rel_schema:
            for item in rel_schema:
                print(f"  {item['relType']}.{item['propertyName']}: {item['propertyTypes']}")
        else:
            print("  (Schema introspection not available)")

get_detailed_schema(driver)

Received notification from DBMS server: <GqlStatusObject gql_status='01N62', status_description='warn: procedure or function execution warning. Execution of the procedure db.schema.nodeTypeProperties() generated the warning The field `propertyTypes` will change output format in the next major version.', position=<SummaryInputPosition line=2, column=17, offset=17>, raw_classification='GENERIC', classification=<NotificationClassification.GENERIC: 'GENERIC'>, raw_severity='WARNING', severity=<NotificationSeverity.WARNING: 'WARNING'>, diagnostic_record={'_classification': 'GENERIC', '_severity': 'WARNING', '_position': {'offset': 17, 'line': 2, 'column': 17}, 'OPERATION': '', 'OPERATION_CODE': '0', 'CURRENT_SCHEMA': '/'}> for query: '\n                CALL db.schema.nodeTypeProperties()\n                YIELD nodeType, propertyName, propertyTypes, mandatory\n                RETURN nodeType, propertyName, propertyTypes, mandatory\n            '
Received notification from DBMS server: <GqlSt

=== Detailed Schema for Text2Cypher ===

Node Types and Properties:
  :`GraphNode`:`module`.name: ['String'] *
  :`GraphNode`:`module`.uid: ['String'] *
  :`GraphNode`:`module`.parent_module: ['String'] *
  :`GraphNode`:`module`.module_name: ['String'] *
  :`GraphNode`:`module`.description: ['String'] *
  :`GraphNode`:`module`.dependencies: ['String'] *
  :`GraphNode`:`module`.source_file: ['String'] *
  :`GraphNode`:`module`.child_modules: ['String'] *
  :`GraphNode`:`module`.main_jsp: ['String'] *
  :`GraphNode`:`module`.base_url: ['String'] *
  :`GraphNode`:`module`.main_action: ['String'] *
  :`GraphNode`:`screen`.name: ['String'] *
  :`GraphNode`:`screen`.uid: ['String'] *
  :`GraphNode`:`screen`.parent_module: ['String'] *
  :`GraphNode`:`screen`.description: ['String'] *
  :`GraphNode`:`screen`.has_search: ['Boolean']
  :`GraphNode`:`screen`.main_jsp: ['String']
  :`GraphNode`:`screen`.title: ['String'] *
  :`GraphNode`:`screen`.has_filter: ['Boolean']
  :`GraphNode`:`screen`.re

## 6. Run Custom Cypher Query

In [10]:
def run_query(driver, query, params=None):
    """Run a custom Cypher query."""
    with driver.session() as session:
        result = session.run(query, params or {})
        data = result.data()
        print(f"Query returned {len(data)} results\n")
        for i, record in enumerate(data, 1):
            print(f"Result {i}:")
            pprint(record)
            print()
        return data

# Example: Find nodes with specific uid
query = """
MATCH (n:GraphNode)
RETURN n.uid as uid, n.name as name, labels(n) as labels
LIMIT 5
"""

run_query(driver, query)

Query returned 5 results

Result 1:
{'labels': ['GraphNode', 'screen'],
 'name': 'After Contract Creation Screen',
 'uid': 'screen:after_contract_creation'}

Result 2:
{'labels': ['GraphNode', 'screen'],
 'name': 'Multiple Order Selection Screen',
 'uid': 'screen:multiple_order_selection'}

Result 3:
{'labels': ['GraphNode', 'screen'],
 'name': 'Contract Modification Screen',
 'uid': 'screen:contract_modification'}

Result 4:
{'labels': ['GraphNode', 'screen'],
 'name': 'Additional Contract Creation Screen',
 'uid': 'screen:additional_contract_creation'}

Result 5:
{'labels': ['GraphNode', 'screen'],
 'name': 'Contract Details Screen',
 'uid': 'screen:contract_details'}



[{'uid': 'screen:after_contract_creation',
  'name': 'After Contract Creation Screen',
  'labels': ['GraphNode', 'screen']},
 {'uid': 'screen:multiple_order_selection',
  'name': 'Multiple Order Selection Screen',
  'labels': ['GraphNode', 'screen']},
 {'uid': 'screen:contract_modification',
  'name': 'Contract Modification Screen',
  'labels': ['GraphNode', 'screen']},
 {'uid': 'screen:additional_contract_creation',
  'name': 'Additional Contract Creation Screen',
  'labels': ['GraphNode', 'screen']},
 {'uid': 'screen:contract_details',
  'name': 'Contract Details Screen',
  'labels': ['GraphNode', 'screen']}]

## 7. Test Relationship Patterns

In [11]:
def explore_relationships(driver, limit=5):
    """Explore relationship patterns in the graph."""
    with driver.session() as session:
        result = session.run("""
            MATCH (a)-[r]->(b)
            RETURN 
                labels(a) as source_labels,
                type(r) as relationship,
                labels(b) as target_labels,
                count(*) as count
            ORDER BY count DESC
            LIMIT $limit
        """, limit=limit).data()
        
        print("=== Common Relationship Patterns ===")
        for item in result:
            source = ":".join(item['source_labels'])
            target = ":".join(item['target_labels'])
            print(f"  ({source})-[{item['relationship']}]->({target}): {item['count']} times")

explore_relationships(driver)

=== Common Relationship Patterns ===
  (GraphNode:function)-[uses_table]->(GraphNode:database_table): 63 times
  (GraphNode:sql_query)-[accesses_table]->(GraphNode:database_table): 57 times
  (GraphNode:form)-[HAS_FIELD]->(GraphNode:form_field): 19 times
  (GraphNode:function)-[uses_query]->(GraphNode:sql_query): 17 times
  (GraphNode:sql_query)-[executed_by]->(GraphNode:dao): 17 times


## 9. Close Connection

## 8. Test Text2Cypher Retriever

Test the Neo4j GraphRAG Text2Cypher retriever with natural language queries.

In [ ]:
# Setup LLM - uses your NXCHAT_API configuration
# Make sure OPENAI_API_KEY and optionally OPENAI_API_BASE are set in your .env
import os

# If using custom OpenAI endpoint, set these
if os.getenv("NXCHAT_API"):
    os.environ["OPENAI_API_BASE"] = os.getenv("NXCHAT_API")
if os.getenv("NXCHAT_API_KEY"):
    os.environ["OPENAI_API_KEY"] = os.getenv("NXCHAT_API_KEY")

# Create LLM instance
llm = OpenAILLM(
    model_name="qwen3-235b-a22b",  # or your custom model
    base_url = os.getenv("NXCHAT_API"),
    model_params={
        "temperature": 0.0,  # More deterministic
    }
)

print(llm)

print("✓ LLM configured")


✓ LLM configured


<coroutine object BaseOpenAILLM.ainvoke at 0x755300bfef40>

In [26]:
# Create Text2CypherRetriever
# Schema will be auto-fetched from Neo4j when set to None
retriever = Text2CypherRetriever(
    driver=driver,
    llm=llm,
    neo4j_schema=None,  # Will auto-fetch schema from database
    examples=[
        # Optional: Add few-shot examples to improve query generation
        # "Question: How many nodes are there? Cypher: MATCH (n) RETURN count(n) as count",
        # "Question: Show me all forms. Cypher: MATCH (n:Form) RETURN n LIMIT 10",
    ],
)

print("✓ Text2CypherRetriever created successfully")

✓ Text2CypherRetriever created successfully


In [28]:
# Test with a natural language question
question = "What is the current existing function  in the database?"

print(f"Question: {question}\n")
print("Sending to Text2Cypher retriever...")

# Search will:
# 1. Convert the question to Cypher using LLM
# 2. Execute the generated Cypher query
# 3. Return results
result = retriever.search(query_text=question)

print("\n" + "="*60)
print("RESULT:")
print("="*60)
pprint(result)

Question: What is the current existing function  in the database?

Sending to Text2Cypher retriever...

RESULT:
RetrieverResult(items=[RetrieverResultItem(content="<Record f.uid='function:list_initialization' f.name='List Initialization Feature' f.description='Initializes and displays contract list screen with all necessary data and permissions'>", metadata=None), RetrieverResultItem(content="<Record f.uid='function:contract_deletion' f.name='Contract Deletion Feature' f.description='Performs logical deletion of selected contract and updates related data'>", metadata=None), RetrieverResultItem(content="<Record f.uid='function:new_contract_creation' f.name='New Contract Creation Feature' f.description='Transitions to contract type selection screen for new contract creation'>", metadata=None), RetrieverResultItem(content="<Record f.uid='function:additional_contract_creation' f.name='Additional Contract Creation Feature' f.description='Transitions to additional contract creation screen'>"

### Understanding the Result Object

The retriever returns a structured result object. Let's extract the key components:

In [31]:
# Inspect the result object structure
print("Result Type:", type(result))
print("\nResult Attributes:")
for attr in dir(result):
    if not attr.startswith('_'):
        print(f"  - {attr}")

# Check if it has common attributes
print("\n" + "="*60)
print("EXTRACTING INFORMATION FROM RESULT")
print("="*60)

# Method 1: Try common attribute names
if hasattr(result, 'items'):
    print("\n✓ Result has 'items' attribute (Pydantic model)")
    print(f"Type: {type(result.items)}")
    print(f"Content: {result.items}")

if hasattr(result, 'content'):
    print("\n✓ Result has 'content' attribute")
    print(f"Content: {result.content}")

# Method 2: Try model_dump if it's a Pydantic model
if hasattr(result, 'model_dump'):
    print("\n✓ Result is a Pydantic model, dumping to dict:")
    result_dict = result.model_dump()
    pprint(result_dict)

Result Type: <class 'neo4j_graphrag.types.RetrieverResult'>

Result Attributes:
  - construct
  - copy
  - dict
  - from_orm
  - items
  - json
  - metadata
  - model_computed_fields
  - model_config
  - model_construct
  - model_copy
  - model_dump
  - model_dump_json
  - model_extra
  - model_fields
  - model_fields_set
  - model_json_schema
  - model_parametrized_name
  - model_post_init
  - model_rebuild
  - model_validate
  - model_validate_json
  - model_validate_strings
  - parse_file
  - parse_obj
  - parse_raw
  - schema
  - schema_json
  - update_forward_refs
  - validate

EXTRACTING INFORMATION FROM RESULT

✓ Result has 'items' attribute (Pydantic model)
Type: <class 'list'>
Content: [RetrieverResultItem(content="<Record f.uid='function:list_initialization' f.name='List Initialization Feature' f.description='Initializes and displays contract list screen with all necessary data and permissions'>", metadata=None), RetrieverResultItem(content="<Record f.uid='function:contract_d

In [33]:
# Extract the answer from the result
def extract_answer_from_result(result):
    """
    Extract the natural language answer from Text2CypherRetriever result.
    
    Returns: (answer, cypher_query, raw_data)
    """
    answer = None
    cypher_query = None
    raw_data = None
    
    # For Pydantic models (neo4j-graphrag v0.5+)
    if hasattr(result, 'model_dump'):
        result_dict = result.model_dump()
        
        # Extract items (query results)
        raw_data = result_dict.get('items', [])
        
        # Extract generated cypher
        metadata = result_dict.get('metadata', {})
        cypher_query = metadata.get('cypher', None)
        
        # The answer needs to be constructed from the data
        # neo4j-graphrag may not always provide a formatted answer
        if raw_data:
            # Format the raw data into a readable answer
            if len(raw_data) == 1 and isinstance(raw_data[0], dict) and len(raw_data[0]) == 1:
                # Single value result (like a count)
                answer = str(list(raw_data[0].values())[0])
            else:
                # Multiple results - format as list
                answer = f"Found {len(raw_data)} results:\n"
                for i, item in enumerate(raw_data[:10], 1):  # Limit to first 10
                    answer += f"\n{i}. {item}"
                if len(raw_data) > 10:
                    answer += f"\n... and {len(raw_data) - 10} more"
        else:
            answer = "No results found"
    
    # For dictionary results
    elif isinstance(result, dict):
        answer = result.get('answer', result.get('response', None))
        cypher_query = result.get('cypher', result.get('query', None))
        raw_data = result.get('data', result.get('items', result.get('results', [])))
    
    # For object with attributes
    else:
        if hasattr(result, 'items'):
            raw_data = result.items
        if hasattr(result, 'metadata'):
            cypher_query = result.metadata.get('cypher') if isinstance(result.metadata, dict) else None
    
    return answer, cypher_query, raw_data

# Extract information
answer, cypher_query, raw_data = extract_answer_from_result(result)

print("="*60)
print("EXTRACTED INFORMATION")
print("="*60)

if cypher_query:
    print("\n📝 Generated Cypher Query:")
    print("-" * 60)
    print(cypher_query)

if answer:
    print("\n💬 Answer:")
    print("-" * 60)
    print(answer)

if raw_data:
    print("\n📊 Raw Data (first 5 items):")
    print("-" * 60)
    pprint(raw_data[:5] if isinstance(raw_data, list) else raw_data)
    if isinstance(raw_data, list) and len(raw_data) > 5:
        print(f"\n... and {len(raw_data) - 5} more items")

EXTRACTED INFORMATION

📝 Generated Cypher Query:
------------------------------------------------------------
MATCH (f:function) RETURN f.uid, f.name, f.description

💬 Answer:
------------------------------------------------------------
Found 8 results:

1. {'content': "<Record f.uid='function:list_initialization' f.name='List Initialization Feature' f.description='Initializes and displays contract list screen with all necessary data and permissions'>", 'metadata': None}
2. {'content': "<Record f.uid='function:contract_deletion' f.name='Contract Deletion Feature' f.description='Performs logical deletion of selected contract and updates related data'>", 'metadata': None}
3. {'content': "<Record f.uid='function:new_contract_creation' f.name='New Contract Creation Feature' f.description='Transitions to contract type selection screen for new contract creation'>", 'metadata': None}
4. {'content': "<Record f.uid='function:additional_contract_creation' f.name='Additional Contract Creation Fea

### Generate Natural Language Answer

Use the LLM to convert the query results into a natural language answer:

In [34]:
def generate_answer_from_results(question, cypher_query, raw_data, llm):
    """
    Use the LLM to generate a natural language answer from query results.
    """
    # Format the data for the prompt
    data_str = json.dumps(raw_data, indent=2, default=str)
    
    # Create a prompt for answer generation
    prompt = f"""Based on the following information, provide a clear and concise answer to the user's question.

Question: {question}

Cypher Query Executed:
{cypher_query}

Query Results:
{data_str}

Please provide a natural language answer that directly addresses the question. Be specific and use the data provided."""

    # Generate the answer
    response = llm.invoke(prompt)
    
    # Extract text from response
    if hasattr(response, 'content'):
        return response.content
    elif isinstance(response, dict):
        return response.get('content', str(response))
    else:
        return str(response)

# Generate a natural language answer
if raw_data and cypher_query:
    print("Generating natural language answer...")
    nl_answer = generate_answer_from_results(question, cypher_query, raw_data, llm)
    
    print("\n" + "="*60)
    print("🤖 GENERATED NATURAL LANGUAGE ANSWER")
    print("="*60)
    print(nl_answer)
else:
    print("⚠ Cannot generate answer - missing query results or cypher query")

Generating natural language answer...


TypeError: 'NoneType' object is not subscriptable

### Complete Pipeline: Question → Cypher → Results → Answer

Putting it all together in a reusable function:

In [ ]:
def text_to_cypher_pipeline(question, retriever, llm):
    """
    Complete pipeline: Question → Cypher → Execute → Generate Answer
    
    Returns: dict with all intermediate results
    """
    print(f"\n{'='*60}")
    print(f"Question: {question}")
    print('='*60)
    
    # Step 1: Use retriever to generate and execute Cypher
    print("\n[1/3] Generating Cypher query...")
    result = retriever.search(query_text=question)
    
    # Step 2: Extract results
    print("[2/3] Extracting query results...")
    answer, cypher_query, raw_data = extract_answer_from_result(result)
    
    if cypher_query:
        print(f"✓ Generated Cypher:\n{cypher_query}\n")
    
    if raw_data:
        result_count = len(raw_data) if isinstance(raw_data, list) else 1
        print(f"✓ Retrieved {result_count} result(s)")
    
    # Step 3: Generate natural language answer
    print("[3/3] Generating natural language answer...")
    if raw_data and cypher_query:
        nl_answer = generate_answer_from_results(question, cypher_query, raw_data, llm)
    else:
        nl_answer = "Unable to generate answer from the query results."
    
    print("\n" + "="*60)
    print("🎯 FINAL ANSWER")
    print("="*60)
    print(nl_answer)
    print("="*60 + "\n")
    
    return {
        'question': question,
        'cypher_query': cypher_query,
        'raw_data': raw_data,
        'answer': nl_answer,
        'result_object': result
    }

# Test the complete pipeline
test_question = "How many different types of nodes exist in the database?"
pipeline_result = text_to_cypher_pipeline(test_question, retriever, llm)

In [ ]:
# Test with multiple questions using the complete pipeline
test_questions = [
    "What are all the relationship types in the graph?",
    "Show me 3 examples of GraphNode entities",
    "How many total relationships are there?",
    "What properties do GraphNode entities have?",
]

print("\n" + "🔄 TESTING MULTIPLE QUESTIONS" + "\n")

results = []
for q in test_questions:
    try:
        result = text_to_cypher_pipeline(q, retriever, llm)
        results.append(result)
    except Exception as e:
        print(f"❌ Error processing question: {e}")
        import traceback
        traceback.print_exc()

print(f"\n✅ Successfully processed {len(results)}/{len(test_questions)} questions")

In [32]:
# Extract key information from the result
def display_retriever_result(result):
    """Display the Text2Cypher retriever result in a readable format."""
    print("\n" + "="*60)
    print("TEXT2CYPHER RESULT BREAKDOWN")
    print("="*60)
    
    # Try to extract generated Cypher query
    cypher_query = None
    if hasattr(result, 'metadata') and result.metadata:
        cypher_query = result.metadata.get('cypher', None)
    elif hasattr(result, 'query'):
        cypher_query = result.query
    elif isinstance(result, dict):
        cypher_query = result.get('cypher', result.get('query', result.get('generated_cypher', None)))
    
    if cypher_query:
        print("\n📝 Generated Cypher Query:")
        print("-" * 60)
        print(cypher_query)
    
    # Try to extract answer/response
    answer = None
    if hasattr(result, 'answer'):
        answer = result.answer
    elif hasattr(result, 'response'):
        answer = result.response
    elif isinstance(result, dict):
        answer = result.get('answer', result.get('response', result.get('text', None)))
    
    if answer:
        print("\n💬 Answer:")
        print("-" * 60)
        print(answer)
    
    # Try to extract raw data
    data = None
    if hasattr(result, 'data'):
        data = result.data
    elif hasattr(result, 'results'):
        data = result.results
    elif isinstance(result, dict):
        data = result.get('data', result.get('results', result.get('records', None)))
    
    if data:
        print("\n📊 Raw Data:")
        print("-" * 60)
        pprint(data)
    
    print("\n" + "="*60)

display_retriever_result(result)


TEXT2CYPHER RESULT BREAKDOWN

📝 Generated Cypher Query:
------------------------------------------------------------
MATCH (f:function) RETURN f.uid, f.name, f.description



### Try More Questions

Test the retriever with different types of questions:

In [ ]:
# Test different questions
test_questions = [
    "How many nodes are in the database?",
    "Show me 5 GraphNode entities",
    "What relationships exist in the graph?",
    "Find all forms in the system",
]

for i, q in enumerate(test_questions, 1):
    print(f"\n{'='*60}")
    print(f"Question {i}: {q}")
    print('='*60)
    
    try:
        result = retriever.search(query_text=q)
        display_retriever_result(result)
    except Exception as e:
        print(f"❌ Error: {e}")
        import traceback
        traceback.print_exc()

In [ ]:
# Close the driver when done
driver.close()
print("✓ Connection closed")